In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")

In [ ]:
import random
import gc
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve,RocCurveDisplay,ConfusionMatrixDisplay,confusion_matrix,roc_auc_score,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing

In [ ]:
train["target"].value_counts() / train.shape[0]

In [ ]:
cols = ["f" + str(i) for i in range(100)]
# plot the first 32 features 
i = 1
plt.figure()
fig, ax = plt.subplots(8, 4,figsize=(20, 22))
for feature in cols[:32]:
    plt.subplot(8, 4,i)
    sns.histplot(train[feature],color="blue", kde=True,bins=100, label='train_'+feature)
    sns.histplot(test[feature],color="olive", kde=True,bins=100, label='test_'+feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
scaler = StandardScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

In [ ]:
y = train["target"]
preds = np.zeros(test.shape[0])
kf = StratifiedKFold(n_splits=15,random_state=48,shuffle=True)
auc=[]  # list contains auc for each fold
acc=[]  # list contains accuracy for each fold
n=0
for trn_idx, test_idx in kf.split(train[cols],y):
    X_tr,X_val=train[cols].iloc[trn_idx],train[cols].iloc[test_idx]
    y_tr,y_val=y.iloc[trn_idx],y.iloc[test_idx]
    
    model = LogisticRegression(penalty = "l1", solver='liblinear')
    model.fit(X_tr,y_tr)
    preds += model.predict_proba(test[cols])[:,1]/kf.n_splits
    
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]))
    acc.append(accuracy_score(y_val, model.predict(X_val)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    n+=1  

In [ ]:
print(f"mean AUC : {round(np.mean(auc)*100,2)} mean Accuracy : {round(np.mean(acc)*100,2)} ")

In [ ]:
sub["target"] = preds

In [ ]:
sub.to_csv("submission.csv", index = None)